In [90]:
import numpy as np
import pandas as pd
from sklearn import datasets
import seaborn as sns
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
train_data= pd.read_csv("train_data.csv")
test_data= pd.read_csv("test_data.csv")

In [3]:
def check_value_counts(data):
    data_keys = data.keys()
    
    for key in data_keys:
        print(key)
        print(data[key].value_counts())
        print('----------------------------')


In [4]:
def check_missing_values(data):
    print(data.isnull().sum())

In [5]:
check_value_counts(train_data)

Item_Identifier
FDG33    10
FDW13    10
NCL31     9
NCY18     9
FDO19     9
FDX04     9
FDT07     9
FDW49     9
FDQ40     9
FDV60     9
FDX31     9
FDX20     9
NCB18     9
DRN47     9
NCQ06     9
FDU12     9
NCF42     9
FDV38     9
NCI54     9
FDF52     9
FDW26     9
FDD38     9
NCJ30     9
FDP25     9
DRE49     9
FDF56     9
FDG09     9
FDO10     8
FDO37     8
NCY29     8
         ..
NCM42     2
FDH22     2
FDM38     2
FDF38     2
FDP15     2
FDB10     2
FDM16     2
FDE39     2
DRL59     2
FDR57     2
DRC24     2
FDG28     2
FDW10     2
FDR03     2
FDA48     2
FDZ50     2
NCG19     2
FDD48     2
FDU43     2
NCW05     2
NCV18     2
FDT35     1
FDC23     1
FDY43     1
FDK57     1
DRF48     1
FDO33     1
FDQ60     1
FDN52     1
FDE52     1
Name: Item_Identifier, Length: 1559, dtype: int64
----------------------------
Item_Weight
12.150    86
17.600    82
13.650    77
11.800    76
15.100    68
9.300     68
16.700    66
10.500    66
19.350    63
20.700    62
16.000    62
9.800     61
17.70

In [6]:
check_missing_values(train_data)

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64


In [7]:
check_missing_values(test_data)

Item_Identifier                 0
Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64


In [8]:
test_data.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type'],
      dtype='object')

In [10]:
#concat test and train 
df = pd.concat([train_data, test_data],ignore_index=True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [11]:
df.head()

,Item_Fat_Content,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Type,Item_Visibility,Item_Weight,Outlet_Establishment_Year,Outlet_Identifier,Outlet_Location_Type,Outlet_Size,Outlet_Type
0,Low Fat,FDA15,249.8092,3735.1380,Dairy,0.016047,9.30,1999,OUT049,Tier 1,Medium,Supermarket Type1
1,Regular,DRC01,48.2692,443.4228,Soft Drinks,0.019278,5.92,2009,OUT018,Tier 3,Medium,Supermarket Type2
2,Low Fat,FDN15,141.6180,2097.2700,Meat,0.016760,17.50,1999,OUT049,Tier 1,Medium,Supermarket Type1
3,Regular,FDX07,182.0950,732.3800,Fruits and Vegetables,0.000000,19.20,1998,OUT010,Tier 3,NaN,Grocery Store
4,Low Fat,NCD19,53.8614,994.7052,Household,0.000000,8.93,1987,OUT013,Tier 3,High,Supermarket Type1


In [12]:
df.pivot_table(values='Item_Outlet_Sales',index='Outlet_Type')


,Item_Outlet_Sales
Outlet_Type,
Grocery Store,339.828500
Supermarket Type1,2316.181148
Supermarket Type2,1995.498739
Supermarket Type3,3694.038558


In [13]:
df.describe()

,Item_MRP,Item_Outlet_Sales,Item_Visibility,Item_Weight,Outlet_Establishment_Year
count,14204.000000,8523.000000,14204.000000,11765.000000,14204.000000
mean,141.004977,2181.288914,0.065953,12.792854,1997.830681
std,62.086938,1706.499616,0.051459,4.652502,8.371664
min,31.290000,33.290000,0.000000,4.555000,1985.000000
25%,94.012000,834.247400,0.027036,8.710000,1987.000000
50%,142.247000,1794.331000,0.054021,12.600000,1999.000000
75%,185.855600,3101.296400,0.094037,16.750000,2004.000000
max,266.888400,13086.964800,0.328391,21.350000,2009.000000


In [21]:
visibility_mean= df.Item_Visibility.mean()
df=df.replace({'Item_Visibility': {0: visibility_mean}}) 


In [22]:
df.describe()

,Item_MRP,Item_Outlet_Sales,Item_Visibility,Item_Weight,Outlet_Establishment_Year
count,14204.000000,8523.000000,14204.000000,11765.000000,14204.000000
mean,141.004977,2181.288914,0.070034,12.792854,1997.830681
std,62.086938,1706.499616,0.048602,4.652502,8.371664
min,31.290000,33.290000,0.003575,4.555000,1985.000000
25%,94.012000,834.247400,0.033143,8.710000,1987.000000
50%,142.247000,1794.331000,0.062347,12.600000,1999.000000
75%,185.855600,3101.296400,0.094037,16.750000,2004.000000
max,266.888400,13086.964800,0.328391,21.350000,2009.000000


In [39]:
df['Item_Visibility_MeanRatio']= df.Item_Visibility/visibility_mean


14199    0.192713
14200    2.041730
14201    1.049895
14202    0.941722
14203    1.495272
Name: Item_Visibility_MeanRatio, dtype: float64

In [40]:
#Get the first two characters of ID:
df['Item_Type_Combined'] = df['Item_Identifier'].apply(lambda x: x[0:2])
#Rename them to more intuitive categories:
df['Item_Type_Combined'] = df['Item_Type_Combined'].map({'FD':'Food',
                                                             'NC':'Non-Consumable',
                                                             'DR':'Drinks'})
df['Item_Type_Combined'].value_counts()

Food              10201
Non-Consumable     2686
Drinks             1317
Name: Item_Type_Combined, dtype: int64

In [41]:
#Years:
df['Outlet_Years'] = 2013 - df['Outlet_Establishment_Year']
df['Outlet_Years'].describe()

count    14204.000000
mean        15.169319
std          8.371664
min          4.000000
25%          9.000000
50%         14.000000
75%         26.000000
max         28.000000
Name: Outlet_Years, dtype: float64

In [45]:
#Change categories of low fat:
print ('Original Categories:')
print( df['Item_Fat_Content'].value_counts())

print ('\nModified Categories:')
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                             'reg':'Regular',
                                                             'low fat':'Low Fat'})
print (df['Item_Fat_Content'].value_counts())

Original Categories:
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

Modified Categories:
Low Fat    9185
Regular    5019
Name: Item_Fat_Content, dtype: int64


In [47]:
#Mark non-consumables as separate category in low_fat:
df.loc[df['Item_Type_Combined']=="Non-Consumable",'Item_Fat_Content'] = "Non-Edible"
df['Item_Fat_Content'].value_counts()

Low Fat       6499
Regular       5019
Non-Edible    2686
Name: Item_Fat_Content, dtype: int64

In [55]:
train_data.Item_Identifier.value_counts()

FDG33    10
FDW13    10
NCL31     9
NCY18     9
FDO19     9
FDX04     9
FDT07     9
FDW49     9
FDQ40     9
FDV60     9
FDX31     9
FDX20     9
NCB18     9
DRN47     9
NCQ06     9
FDU12     9
NCF42     9
FDV38     9
NCI54     9
FDF52     9
FDW26     9
FDD38     9
NCJ30     9
FDP25     9
DRE49     9
FDF56     9
FDG09     9
FDO10     8
FDO37     8
NCY29     8
         ..
NCM42     2
FDH22     2
FDM38     2
FDF38     2
FDP15     2
FDB10     2
FDM16     2
FDE39     2
DRL59     2
FDR57     2
DRC24     2
FDG28     2
FDW10     2
FDR03     2
FDA48     2
FDZ50     2
NCG19     2
FDD48     2
FDU43     2
NCW05     2
NCV18     2
FDT35     1
FDC23     1
FDY43     1
FDK57     1
DRF48     1
FDO33     1
FDQ60     1
FDN52     1
FDE52     1
Name: Item_Identifier, Length: 1559, dtype: int64

In [70]:
# Once all the exploration is done, lets add all the changes we need to make in a function
def preProcessingData(data,feature_columns):
    data = data[feature_columns]
    #Handling NA Item_Weight 
    data['Item_Weight'].fillna((data['Item_Weight'].mean()), inplace=True)
    #Filling Outlet Size with mode
    data['Outlet_Size'].fillna(data['Outlet_Size'].mode()[0], inplace=True)
    #Some Item visibility was zero
    visibility_mean= data.Item_Visibility.mean()
    data=data.replace({'Item_Visibility': {0: visibility_mean}}) 
    #Creating Broad Category For Products - classified into Food, Non-Consumable, Drinks
    data['Item_Type_Combined'] = np.where(data.Item_Identifier.str.startswith('FD'), 'Food', 
            np.where(data.Item_Identifier.str.startswith('NC'), 'Non-Consumable', 
            np.where(data.Item_Identifier.str.startswith('DR'), 'Drink',0)))
    #Year since the outlet is opened. 
    data['Outlet_Years'] = 2013 - data['Outlet_Establishment_Year']
    
    #Correcting Labels in Item_Fat_Content
    #Change categories of low fat:

    data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                             'reg':'Regular',
                                                             'low fat':'Low Fat'})
    #Mark non-consumables as separate category in low_fat:
    data.loc[data['Item_Type_Combined']=="Non-Consumable",'Item_Fat_Content'] = "Non-Edible"
    return data

In [141]:
#Reading Training, Test Data, separating target variable from training, then cleaning and preprocessing data for ML 
train_data = pd.read_csv("train_data.csv")
test_data = pd.read_csv("test_data.csv")
TargetLabel = train_data['Item_Outlet_Sales']
feature_columns = ['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
                   'Item_Type', 'Item_MRP', 'Outlet_Identifier',
                   'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type','Outlet_Type']
#calling the data cleaning function
preprocessed_test_data = preProcessingData(test_data,feature_columns)
preprocessed_train_data = preProcessingData(train_data,feature_columns)

merged_data = pd.concat([preprocessed_train_data,preprocessed_test_data])




/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [142]:
merged_data.drop('Item_Identifier', axis=1, inplace=True)
preprocessed_train_data.drop('Item_Identifier', axis=1, inplace=True)
preprocessed_test_data.drop('Item_Identifier', axis=1, inplace=True)
merged_data.drop('Outlet_Identifier', axis=1, inplace=True)
preprocessed_train_data.drop('Outlet_Identifier', axis=1, inplace=True)
preprocessed_test_data.drop('Outlet_Identifier', axis=1, inplace=True)
merged_data.drop('Item_Type', axis=1, inplace=True)
preprocessed_train_data.drop('Item_Type', axis=1, inplace=True)
preprocessed_test_data.drop('Item_Type', axis=1, inplace=True)


In [143]:
merged_data.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Years
count,14204.000000,14204.000000,14204.000000,14204.000000,14204.000000
mean,12.792847,0.070034,141.004977,1997.830681,15.169319
std,4.234354,0.048602,62.086938,8.371664,8.371664
min,4.555000,0.003575,31.290000,1985.000000,4.000000
25%,9.300000,0.033143,94.012000,1987.000000,9.000000
50%,12.850000,0.062347,142.247000,1999.000000,14.000000
75%,16.000000,0.094037,185.855600,2004.000000,26.000000
max,21.350000,0.328391,266.888400,2009.000000,28.000000


In [144]:

y=train_data['Item_Outlet_Sales'] # define the target variable (dependent variable) as y

In [145]:
y.describe()

count     8523.000000
mean      2181.288914
std       1706.499616
min         33.290000
25%        834.247400
50%       1794.331000
75%       3101.296400
max      13086.964800
Name: Item_Outlet_Sales, dtype: float64

In [146]:
Numerical_merged_data = pd.get_dummies(merged_data)
processed_train_data = Numerical_merged_data.iloc[:preprocessed_train_data.shape[0],:]
processed_test_data = Numerical_merged_data.iloc[preprocessed_train_data.shape[0]:,:]


In [147]:
processed_test_data.head()


,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Years,Item_Fat_Content_Low Fat,Item_Fat_Content_Non-Edible,Item_Fat_Content_Regular,Outlet_Size_High,Outlet_Size_Medium,...,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Type_Combined_Drink,Item_Type_Combined_Food,Item_Type_Combined_Non-Consumable
0,20.750000,0.007565,107.8622,1999,14,1,0,0,0,1,...,1,0,0,0,1,0,0,0,1,0
1,8.300000,0.038428,87.3198,2007,6,0,0,1,0,1,...,0,1,0,0,1,0,0,0,1,0
2,14.600000,0.099575,241.7538,1998,15,0,1,0,0,1,...,0,0,1,1,0,0,0,0,0,1
3,7.315000,0.015388,155.0340,2007,6,1,0,0,0,1,...,0,1,0,0,1,0,0,0,1,0
4,12.695633,0.118599,234.2300,1985,28,0,0,1,0,1,...,0,0,1,0,0,0,1,0,1,0


In [148]:
processed_test_data.dtypes


Item_Weight                          float64
Item_Visibility                      float64
Item_MRP                             float64
Outlet_Establishment_Year              int64
Outlet_Years                           int64
Item_Fat_Content_Low Fat               uint8
Item_Fat_Content_Non-Edible            uint8
Item_Fat_Content_Regular               uint8
Outlet_Size_High                       uint8
Outlet_Size_Medium                     uint8
Outlet_Size_Small                      uint8
Outlet_Location_Type_Tier 1            uint8
Outlet_Location_Type_Tier 2            uint8
Outlet_Location_Type_Tier 3            uint8
Outlet_Type_Grocery Store              uint8
Outlet_Type_Supermarket Type1          uint8
Outlet_Type_Supermarket Type2          uint8
Outlet_Type_Supermarket Type3          uint8
Item_Type_Combined_Drink               uint8
Item_Type_Combined_Food                uint8
Item_Type_Combined_Non-Consumable      uint8
dtype: object

In [149]:
X_train, X_test, Y_train, Y_test = train_test_split(processed_train_data, y, test_size=0.2)
print( X_train.shape, Y_train.shape)
print (X_test.shape, Y_test.shape)


(6818, 21) (6818,)
(1705, 21) (1705,)


In [150]:
Y_train.describe()

count     6818.000000
mean      2184.546830
std       1707.949374
min         33.290000
25%        843.568600
50%       1799.657400
75%       3094.638400
max      13086.964800
Name: Item_Outlet_Sales, dtype: float64

In [151]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline

In [153]:
pipelines = []
pipelines.append(('LR', Pipeline([('LR',LinearRegression())])))
pipelines.append(('LASSO', Pipeline([('LASSO', Lasso())])))
pipelines.append(('EN', Pipeline([('EN', ElasticNet())])))
pipelines.append(('KNN', Pipeline([('KNN', KNeighborsRegressor())])))
pipelines.append(('CART', Pipeline([('CART', DecisionTreeRegressor())])))
pipelines.append(('GBM', Pipeline([('GBM', GradientBoostingRegressor())])))

results = []
names = []
for name, model in pipelines:
    kfold = KFold(n_splits=10, random_state=21)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='neg_mean_squared_error')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: -1289675.553027 (90914.645518)
LASSO: -1290909.916193 (91472.669032)
EN: -1698943.703121 (104679.176795)
KNN: -1570703.867374 (121784.697707)
CART: -2414530.441173 (178318.535597)
GBM: -1189055.324257 (100750.409845)


In [156]:
from sklearn.metrics import mean_squared_error

scaler = StandardScaler().fit(X_train)
rescaled_X_train = scaler.transform(X_train)
model = GradientBoostingRegressor(random_state=21, n_estimators=400)
model.fit(rescaled_X_train, Y_train)

# transform the validation dataset
rescaled_X_test = scaler.transform(X_test)
predictions = model.predict(rescaled_X_test)
print (mean_squared_error(Y_test, predictions))

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


1176872.216605805


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':


In [157]:
compare = pd.DataFrame({'Prediction': predictions, 'Test Data' : Y_test})
compare.head(10)


,Prediction,Test Data
5131,3905.675253,4837.7028
525,3766.132085,5416.9488
217,850.575711,260.9936
8426,2799.723687,4512.1266
5048,4960.297627,524.6504
1704,2710.245908,1845.5976
2153,-1118.891770,262.9910
4896,2873.172843,4570.0512
6654,3669.581312,4527.4400
1235,2583.540802,3235.7880


In [162]:
#Define target and ID columns:
target = 'Item_Outlet_Sales'
IDcol = ['Item_Identifier','Outlet_Identifier']
from sklearn import metrics
def modelfit(alg, dtrain, dtest, predictors, target, IDcol, filename):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])

    #Perform cross-validation:
    cv_score = cross_validation.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=20, scoring='mean_squared_error')
    cv_score = np.sqrt(np.abs(cv_score))
    
    #Print model report:
    print( "\nModel Report")
    print ("RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(dtrain[target].values, dtrain_predictions)))
    print ("CV Score : Mean - %.4g | Std - %.4g | Min - %.4g | Max - %.4g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
    #Predict on testing data:
    dtest[target] = alg.predict(dtest[predictors])
    
    #Export submission file:
    IDcol.append(target)
    submission = pd.DataFrame({ x: dtest[x] for x in IDcol})
    submission.to_csv(filename, index=False)

In [164]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
predictors = [x for x in X_train.columns if x not in [target]+IDcol]
# print predictors
alg1 = LinearRegression(normalize=True)
modelfit(alg1, X_train, Y_train, predictors, target, IDcol, 'alg1.csv')
coef1 = pd.Series(alg1.coef_, predictors).sort_values()
coef1.plot(kind='bar', title='Model Coefficients')

KeyError: 'Item_Outlet_Sales'